# Unity Catalog: Control de Acceso y Seguridad

## Bloque 1: Intro Unity Catalogo

%md
### 1.1 Rol de Unity Catalog
En Unity Catalog normalmente se sigue **este patrón**:

**Un único metastore** → nivel más alto.

Dentro del metastore → **varios catálogos que segmentan grandes ámbitos**:
  - Por dominio de negocio: finance, sales, hr…
  - O por entorno: dev, test, prod.

Dentro de cada catálogo → **esquemas que organizan áreas más concretas** (ej. dentro de sales: bronze, silver, gold o crm, orders, etc.).

**Dentro de los esquemas → tablas, vistas, funciones, modelos.**

👉 Lo que resuelve UC es tener un metastore centralizado que te permita:

  - Ver todos los catálogos y esquemas disponibles en un solo sitio.
  - Aplicar seguridad y lineage de forma coherente.
  - Evitar que cada departamento levante “su Hive metastore paralelo” que nadie más puede ver.

Aporta: metastore unificado, permisos centralizados, lineage automático, descubrimiento de datos, data sharing seguro.   

%md
### 1.2 Jerarquía y definiciones
- **Metastore**: la “biblioteca” central de metadatos. Describe **qué** objetos existen (tablas, vistas, funciones, modelos), **dónde** están y **con qué propiedades**.  
- **Catálogo**: contenedor de **esquemas**. Suele separar dominios (p. ej., `finance`, `hr`) o **entornos** (`dev`, `test`, `prod`).  
- **Esquema**: contenedor dentro del catálogo. Suele mapear capas funcionales (p. ej., `bronze`, `silver`, `gold`) o subdominios.  
- **Tabla**: estructura con **columnas tipadas** y **filas**. Puede ser *managed* (gestionada por Databricks) o *external* (ruta externa).  
- **Vista**: consulta guardada que proyecta o filtra datos. Útil para **seguridad a nivel de fila o columna**.  
- **Función**: lógica reusable (SQL/UDF) gobernada por UC.  
- **Modelo**: artefacto de ML registrado y gobernado por UC.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/uc-base-1.png?raw=true" style="float: right" width="800px"/> 

**Resumen jerárquico**
```
Metastore → Catálogo → Esquema → (Tabla / Vista / Función / Modelo)
```

- Metastore = biblioteca con todos los “libros”.  
- Workspace = tu aula de Databricks.  
- Enlazar = *“esta aula puede usar los libros de esa biblioteca”*.  
Así, desde tu workspace, **ves y usas** catálogos, esquemas y tablas descritos en el metastore.

## Bloque 2: Control de Accesos y Seguridad en UC

Gestión de un Metastore, Tipos de Grupos y Tipos de Accesos

%md
- Ahora veremos de manera práctica la creación de grupos.
- Patrón típico: **grupo** (p. ej., *Data Engineers* o *Gobernanza*) como owner de catálogos.  
- **Permisos** frecuentes: `USE CATALOG/SCHEMA`, `SELECT`, `INSERT`, `MODIFY`, `CREATE`, `ALL PRIVILEGES`.  
- Aplica **mínimo privilegio** desde el principio.

### 2.1 Creamos la Jerarquía base de UC (Catalogo-Esquema-Volumnen)

> Copia el siguiente contenido en una celda de codigo

```python
spark.sql(f"CREATE CATALOG IF NOT EXISTS sesion3")
sql(f"USE CATALOG sesion3")

sql(f"CREATE SCHEMA IF NOT EXISTS retail_ventas")
```

%md
### 2.2 Gestión de usuarios y grupos en Unity Catalog

La **gestión de identidades** es clave para aplicar políticas de seguridad en Databricks.  
En lugar de asignar permisos a usuarios individuales, se recomienda trabajar con **grupos**, lo que simplifica y hace más escalable la administración.

### 2.3  Tipos de Grupos: Workspace vs Account
- **Workspace**: los grupos/usuarios creados solo existen dentro de un workspace. Sirven para notebooks o jobs, **pero no son válidos en Unity Catalog**.  
- **Account**: los grupos/usuarios definidos a nivel de cuenta (manual o sincronizados desde Azure AD, Okta, etc.) son los que **reconoce Unity Catalog** y permiten aplicar gobernanza centralizada de datos.  

👉 **Regla general**: para producción y para usar todas las capacidades de UC, siempre define usuarios y grupos a nivel de cuenta.


### 2.4 ¿Hay herencia de permisos en Unity Catalog?

**No existe herencia automática hacia abajo.**  
En Unity Catalog, asignar un permiso en un nivel superior **(catálogo)** no concede automáticamente acceso a los esquemas ni a las tablas que contiene.

Para acceder a una tabla, es obligatorio que el usuario/grupo tenga:

- `USAGE` en el **catálogo**
- `USAGE` en el **esquema**
- `SELECT` (u otro permiso específico) en la **tabla o vista**

Si falta uno solo de esos permisos, el acceso será bloqueado — aunque se haya concedido un permiso en un nivel inferior o superior.

➡️ UC **evalúa jerárquicamente**, pero **no hereda permisos por defecto**. Cada nivel debe ser concedido explícitamente.


### 2.5 Ejemplo guiado

#### 2.5.1 Creamos 2 grupos a nivel de cuenta

Haz clic en el ícono de tu usuario en la esquina superior derecha de la pantalla  
y selecciona la opción **Settings**.


<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm1.png" width="1600px"/>

Dentro de la pantalla **Settings**, haz clic en **Identity and access**  
y luego, en la sección **Manage**, selecciona **Groups**.


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm2.png" width="1600px"/>

Haz clic en **Add group** y luego selecciona **Add new** para crear un nuevo grupo.


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm3.png" width="1600px"/>

Crea dos grupos con los siguientes nombres:

🔹 `UC_Data_Analyst`  
🔹 `UC_Data_Engineer`


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm4.png" width="800px"/>

#### 2.5.2 Creamos un nuevo usuario a nivel de cuenta

Dirígete nuevamente a **Identity and access**, pero esta vez a la sección **Users  → Manage**.


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm5.png" width="1600px"/>

Al igual que en la creación de grupos, continúa el proceso para **crear un nuevo usuario**.  
Utiliza un **segundo correo electrónico** que tengas disponible.


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm6.png" width="800px"/>

#### 2.5.3 Agregamos el nuevo usuario a el grupo UC_Data_Engineer

Vuelve nuevamente a la sección **Manage → Groups**.


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm7.png" width="1600px"/>

Haz clic en el grupo **UC_Data_Engineer** y luego selecciona **Add members**  
para añadir al nuevo usuario a ese grupo.


%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm8.png" width="1600px"/>

%md
<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_perm9.png" width="800px"/>

#### 2.5.4 Comparativa entre Usuarios y Grupos en Workspace vs Cuenta

La siguiente tabla resume las diferencias clave entre los usuarios y grupos gestionados a nivel de workspace y los gestionados a nivel de cuenta. Es fundamental entender estas diferencias para diseñar una estrategia de seguridad robusta y compatible con Unity Catalog.

| Característica                               | Workspace User/Group     | Account User/Group      |
|----------------------------------------------|--------------------------|-------------------------|
| Visibles solo en un workspace                | ✅ Sí                    | ❌ No (son globales)   |
| Útiles para permisos en notebooks, jobs, etc.| ✅ Sí                    | ✅ Sí                  |
| Se usan para Unity Catalog (datos)           | ❌ No                    | ✅ Sí                  |
| Gobernanza centralizada de datos             | ❌ No                    | ✅ Sí                  |
| Accesibles desde todos los workspaces        | ❌ No                    | ✅ Sí                  |
| Gestión desde Workspace UI                   | ✅ Sí                    | ❌ No (solo desde Account Console o SCIM API) |
| Sincronizables desde IdP (Azure AD, Okta)    | ❌ No                    | ✅ Sí                  |
| Ideal para producción                        | ❌ No                    | ✅ Sí                  |

Como regla general, para entornos productivos y para aprovechar todas las capacidades de Unity Catalog, utiliza siempre usuarios y grupos de cuenta.

### 2.6 Tipos de permisos

#### 2.6.1 A nivel de Metastore / Catálogo / Esquema

- ``USE CATALOG`` / ``USE SCHEMA`` → permite acceder al catálogo/esquema.
- ``CREATE`` → permite crear objetos (esquemas, tablas, vistas, funciones, etc.).
- ``MODIFY`` → modificar objetos existentes.
- ``ALL PRIVILEGES`` → acceso total al objeto

#### 2.6.2 A nivel de Tablas y Vistas

- ``SELECT`` → lectura de datos.
- ``INSERT`` → insertar nuevas filas.
- ``UPDATE`` → actualizar filas existentes.
- ``DELETE`` → eliminar filas.
- ``TRUNCATE`` → vaciar tabla.
- ``ALL PRIVILEGES`` → todos los anteriores.

%md
**Permisos base de ``USE``**

Para que un grupo pueda ver/usar objetos que cuelgan de un catálogo o esquema, primero necesita USE.

%md
##### Vamos a dar accesos a nuestro grupo

> Copia el siguiente contenido en una celda

```python
GRANT ALL PRIVILEGES ON CATALOG sesion3 TO `UC_Data_Engineer`;
```

%md
##### Podemos visualizar facilmente los accesos actuales de un schema

In [0]:
display(spark.sql(f"SHOW GRANTS ON SCHEMA sesion3.retail_ventas"))

Principal,ActionType,ObjectType,ObjectKey
UC_Data_Engineer,ALL PRIVILEGES,CATALOG,sesion3


#### 2.6.3 Demo de accesos a otros usuarios

##### 2.6.3.1 Creamos un nuevo catalogo, schemas para el ejemplo

> Copia el siguiente contenido en una celda

```python
# =========================
# Creacion de Catalogo>Esquema
# =========================
catalog_perm = "permissions"
schema_bz, schema_sv, schema_gd = "bronze", "silver", "gold"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_perm}")
for s in (schema_bz, schema_sv, schema_gd):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_perm}.{s}")
```

##### 2.6.3.2 creamos algunas tablas e insertaremos algunos datos en ellas

> Copia el siguiente contenido en una celda


```python
# =========================
# Datos de demo
# =========================
# Bronze
spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_bz}.raw_sales (id INT, raw_line STRING)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_bz}.raw_sales VALUES (1,'1,blade,10'),(2,'2,tower,5')")

# Silver (EU/NA)
spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_sv}.sales_EU (id INT, item STRING, qty INT, region STRING)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_sv}.sales_EU VALUES (1,'blade',10,'FR'),(2,'tower',5,'ES')")

spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_sv}.sales_NA (id INT, item STRING, qty INT, region STRING)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_sv}.sales_NA VALUES (1,'blade',8,'US'),(2,'tower',6,'CA')")

# Gold
spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_gd}.sales_summary (item STRING, total_qty INT)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_gd}.sales_summary VALUES ('blade',18),('tower',11)")
```

%md
## /!\ Dificultad para entrar al workspace compartido /!\

**``una vez dado el acceso les llegara un email a su cuenta para aceptar la participacion``**

Es posible que al aceptar les lleve directamente a mi cuenta. Hay que hacer Log out. Salir del todo

Al entrar nuevamente les saldra para elegir 2 perfiles/workspaces la nueva opcion tendra el nombre del que ha invitado en la participacion.

##### 2.6.3.3 Damos permisos a los usuarios del grupo Data Engineer

en este caso estamos dando acceso de visualizar el catalogo permissions y los 3 esquemas que contiene

> Copia el siguiente contenido en una celda

```python
# =========================
# Grants básicos de navegación
# =========================

#ambos perfiles tendran acceso al catalogo

spark.sql(f"GRANT USE CATALOG ON CATALOG {catalog_perm} TO UC_Data_Engineer")


#perfil engineer tendra acceso a schema bronze, silver y gold
for s in (schema_bz, schema_sv, schema_gd):
    spark.sql(f"GRANT USE SCHEMA ON SCHEMA {catalog_perm}.{s} TO UC_Data_Engineer")
```

Damos adicionalmente capacidad de escritura en los schemas bronze y silver. Ahora los usuarios de dentro de ese grupo podrán crear tablas dentro de esos schemas

> Copia el siguiente contenido en una celda

```python
# =========================
# Permisos de esquema (Engineers construyen en Bronze/Silver)
# =========================

#Damos a ingenieros permiso de escritura de tabla
for s in (schema_bz, schema_sv):
    # En PM 1.0, CREATE TABLE en el SCHEMA habilita crear tablas y vistas en ese schema.
    spark.sql(f"GRANT CREATE TABLE ON SCHEMA {catalog_perm}.{s} TO UC_Data_Engineer")

```

Damos acceso a las tablas sales_EU y sales_Na en silver y a la tabla sales_summary de gold

> Copia el siguiente contenido en una celda

```python
spark.sql(f"GRANT SELECT, MODIFY ON TABLE {catalog_perm}.{schema_sv}.sales_EU TO UC_Data_Engineer")
spark.sql(f"GRANT SELECT, MODIFY ON TABLE {catalog_perm}.{schema_sv}.sales_NA TO UC_Data_Engineer")
spark.sql(f"GRANT SELECT, MODIFY ON TABLE {catalog_perm}.{schema_gd}.sales_summary TO UC_Data_Engineer")
```

##### 2.6.3.4 Workspace de Ingeniero ejecuta:


Vamos ahora a comprobar la información disponible del usuario alocado en el grupo UC_Data_Engineer

desde el workspace del ingeniero ejecuta el siguiente codigo

> Copia el siguiente contenido en una celda en el workspace del UC_Data_Engineer

```python
catalog_perm = "permissions"
schema_bz, schema_sv, schema_gd = "bronze", "silver", "gold"

lab_table = "sales_lab"
lab_view  = "vw_items"

# Crear tabla en Silver
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_perm}.{schema_sv}.{lab_table} (
  id INT,
  item STRING,
  qty INT,
  region STRING
)
""")

# Insert/Update/Delete permitidos
spark.sql(f"INSERT INTO {catalog_perm}.{schema_sv}.{lab_table} VALUES (100,'hub',3,'EU'),(101,'blade',2,'ES')")
spark.sql(f"UPDATE {catalog_perm}.{schema_sv}.{lab_table} SET qty = 99 WHERE id = 100")
spark.sql(f"DELETE FROM {catalog_perm}.{schema_sv}.{lab_table} WHERE id = 101")
```

##### 2.6.3.5 Revocamos acceso a el schema gold


Ahora desde nuestro workspace principal vamos a revocar el acceso del usuario de UC_Data_Engineer al schema gold

> Copia el siguiente contenido en una celda

```python
spark.sql(f"REVOKE SELECT ON TABLE {catalog_perm}.{schema_gd}.sales_summary FROM UC_Data_Engineer")
```

#### 2.6.4 Ver accesos de un perfil sobre un catalogo o schema

Podemos comprobar fácilmente todos los accesos que tiene un grupo.

> Copia el siguiente contenido en una celda

```python
SELECT *
FROM permissions.information_schema.schema_privileges
WHERE grantee = 'UC_Data_Engineer';
```

### 2.7	Seguridad fila/columna

- Row-level con vistas.
- Column masking con funciones.
- Ejercicio: vista filtrada por región, vista que oculta email.

#### 2.7.0  Crear Estructura Catalogo, Esquema y Volumen

> Copia el siguiente contenido en una celda

```python
# =========================
# Vars (ajusta usuarios y paths)
# =========================
catalog_ej2 = "chairman"
schema_bz, schema_sv, schema_gd = "bronze", "silver", "gold"
schema_sec = "security"  

spark.sql(f"DROP CATALOG IF EXISTS {catalog_ej2} CASCADE")


spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_ej2}")
for s in (schema_bz, schema_sv, schema_gd,schema_sec):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_ej2}.{s}")

##Crea el volume para aterrizar ficheros
spark.sql(f"CREATE VOLUME IF NOT EXISTS chairman.bronze.flights_vol")
```

#### 2.7.1 Traemos datos desde el repositorio 

> Copia el siguiente contenido en una celda

```pythoh
curl -L https://raw.githubusercontent.com/jmartinezceste/251101Course_UC_Ceste/refs/heads/main/UC_Course_Sesion2/data/flights.csv -o /Volumes/chairman/bronze/flights_vol/flights.csv
```

#### 2.7.2 Creamos DF y Tabla delta en silver

> Copia el siguiente contenido en una celda

```python
csv_path = f"/Volumes/{catalog_ej2}/{schema_bz}/flights_vol/flights.csv"
df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(csv_path))

display(df.limit(2))
```

> Copia el siguiente contenido en una celda

```python
# Normaliza tipos (por si inferSchema deja algo como string numérico)
from pyspark.sql.functions import col
df = (df
      .withColumn("fare_eur", col("fare_eur").cast("decimal(10,2)"))
      .withColumn("flight_id", col("flight_id").cast("string")))

table_flights = f"{catalog_ej2}.{schema_sv}.flights"

df.write.mode("overwrite").saveAsTable(table_flights)
```

#### 2.7.3 Exploracion Tabla

Vemos que las columnas, origines, destinos, nombre, email, tarjeta credito.... 

El ``objetivo va a ser``:
- ocultar todo lo que sea mercado = NA para ingenieros 
- encriptar la informacion sensible (email, tarjeta credito...)

todo ello sin crear nuevas tablas. dependiendo de que usuario accede, tendra disponible una informacion u otra

> Copia el siguiente contenido en una celda

```python
display(
    spark.sql(
        "select * from chairman.silver.flights limit 3"
    )
)


display(
    spark.sql(
        "select distinct(market) from chairman.silver.flights"
    )
)

```

#### 2.7.4 Damos accesos a Nuevo Catalogo y esquemas

Damos accesos a catalogop y esquemas a ingenieros.

En este punto solo tendran acceso a ver el catalogo y esquema. Aun no les habremos dado acceso a la tabla, por lo que les aparecera el schema vacio

> Copia el siguiente contenido en una celda

```python
# =========================
# Grants mínimos de navegación
# =========================

spark.sql(f"GRANT USE CATALOG ON CATALOG {catalog_ej2} TO UC_Data_Engineer")
spark.sql(f"GRANT USE SCHEMA  ON SCHEMA  {catalog_ej2}.{schema_sv} TO UC_Data_Engineer")
spark.sql(f"GRANT USE SCHEMA  ON SCHEMA  {catalog_ej2}.{schema_sec} TO UC_Data_Engineer")
```

#### 2.7.5 Row level access control 


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/acls/table_uc_rls.png?raw=true" width="200" style="float: right; margin-top: 20; margin-right: 20" alt="databricks-demos"/>

La seguridad a nivel de fila le permite ocultar automáticamente un subconjunto de sus filas en función de quién intente consultarlas, sin tener que mantener copias separadas de sus datos.

Un caso de uso típico sería filtrar las filas en función de su país o unidad de negocio: solo verá los datos (transacciones financieras, pedidos, información de clientes...) pertenecientes a su región, lo que le impedirá acceder a todo el conjunto de datos.

💡 Aunque este filtro se puede aplicar a nivel de usuario/principal, se recomienda implementar políticas de acceso utilizando grupos.
<br style="clear: both"/>

##### 2.7.5.1 Definir la regla de acceso

Para declarar una regla de control de acceso, deberá crear una función SQL que devuelva un valor **booleano**.
Unity Catalog ocultará la fila si la función devuelve `False`.

Dentro de la función SQL, puede definir diferentes condiciones e implementar una lógica compleja para crear este valor de retorno booleano. (Por ejemplo:  `IF(condición)-THEN(vista)-ELSE`)

Aquí, aplicaremos la siguiente lógica:

- Tipo: función SQL.
- input: recibe el valor de la columna market y devuelve TRUE/FALSE.
- Lógica:
  - Si el usuario actual (CURRENT_USER()) está en la lista de engineers → TRUE (ve todas las filas).
  - Si no es engineer → solo TRUE cuando market='EU'.

La funcion queda almacenada en el schema security

Nos colocamos a nosotros como valor True para que no se nos encripte

> Copia el siguiente contenido en una celda

```python
CREATE OR REPLACE FUNCTION chairman.security.rf_only_eu_or_engineers(market STRING)
RETURNS BOOLEAN
RETURN
  CASE
    -- Si el usuario está en la lista de engineers → acceso total (TRUE SIEMPRE)
    WHEN CURRENT_USER() IN ('251104javierceste@gmail.com') THEN TRUE

    -- Si NO es engineer → solo permitimos EU
    WHEN market = 'EU' THEN TRUE

    -- En cualquier otro caso → DENEGADO
    ELSE FALSE
  END;
```

##### 2.7.5.2 Aplicar la regla de acceso

Una vez declarada nuestra función de regla, solo queda aplicarla a una tabla y verla en acción.
Basta con un simple `SET ROW FILTER` seguido de una llamada a la función.

**Nota: si esto falla, asegúrese de que está utilizando un clúster compartido.**

> Copia el siguiente contenido en una celda

```python
%sql
ALTER TABLE chairman.silver.flights 
SET ROW FILTER chairman.security.rf_only_eu_or_engineers
ON (market);
```

> Copia el siguiente contenido en una celda

```python
%sql
select distinct(market)
from chairman.silver.flights
```

##### 2.7.4.3 Damos accesos a la tabla

> Copia el siguiente contenido en una celda

```python
# Engineers: lectura + modificación (INSERT/UPDATE/DELETE/TRUNCATE/MERGE)

spark.sql(f"GRANT SELECT, MODIFY ON TABLE chairman.silver.flights TO UC_Data_Engineer")
```

%md
###### Resultado de la política de seguridad

**Engineer ejecuta → `SELECT * FROM flights`**

| Fila                 | Evaluación función                       | Resultado |
|----------------------|-------------------------------------------|-----------|
| (market = 'NA', ...) | FALSE (no EU) y tampoco en engineers   | ❌ Filtrada |
| (market = 'EU', ...) | TRUE (es EU)                          | ✅ Visible |

<br>

**Yo ejecuto lo mismo → `SELECT * FROM flights`**

| Fila                 | Evaluación función                       | Resultado |
|----------------------|-------------------------------------------|-----------|
| Cualquier market      | CURRENT_USER() IN mi perfil  → TRUE     | ✅ Todas visibles |

Si ahora desde el workspace del usuario del grupo de UC_Data_Engineer hacemos un:


```python
%sql
select *
from chariman.silver.flights
```

veremos unicamente los valores donde Market = EU y no los valores de Market = NA

#### 2.7.5 Column Level access control 

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/acls/table_uc_cls.png?raw=true" width="200" style="float: right; margin-top: 20; margin-right: 20; margin-left: 20" alt="databricks-demos"/>

Del mismo modo, el control de acceso a nivel de columna le ayuda a ocultar o anonimizar los datos que se encuentran en determinadas columnas de su tabla, dependiendo del usuario o del servicio principal que intente acceder a ellos. Esto se utiliza normalmente para ocultar o eliminar información confidencial de sus usuarios finales (correo electrónico, número de la seguridad social, etc.).

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=1330931038747594&notebook=%2F01-Row-Column-access-control&demo_name=uc-01-acl&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fuc-01-acl%2F01-Row-Column-access-control&version=1">

##### 2.7.5.1 Definir la regla de acceso

Aqui vamos a definir 2 normas de acceso a la información.

Enmascararemos tanto la información sensible de la columna email como de la columna credit_card.

Para el primer caso haremos que solo los miembros del grupo UC_Data_Engineer vean la informacion enmascarada

Para el segundo tanto ellos como nosotros mismo la veremos enmascarada

> Copia el siguiente contenido en una celda

```python
CREATE OR REPLACE FUNCTION chairman.security.mask_email(e STRING)
RETURNS STRING
LANGUAGE SQL
RETURN CASE
    WHEN e IS NULL THEN NULL
    WHEN is_member('UC_Data_Engineer') THEN CONCAT(SUBSTR(e, 1, 2), '***@***')  -- Engineer ve mascarado
    ELSE e                                                                     -- El resto ve el email completo
END;
```

> Copia el siguiente contenido en una celda

```python
CREATE OR REPLACE FUNCTION chairman.security.mask_credit_card(cc STRING)
RETURNS STRING
RETURN CASE
    WHEN cc IS NULL THEN NULL                        
    ELSE CONCAT('**** **** **** ', RIGHT(cc, 4))     -- Todos ven solo últimos 4
END;
```

##### 2.7.5.2 Aplicamos la política de enmascaracion de columnas a la tabla

aplicamos tanto las funciones de enmascaramiento como la de filtrado de filas para crear una vista

> Copia el siguiente contenido en una celda

```python
-- Vista para Analysts: solo EU + PII enmascarada con UDFs dinámicas
CREATE OR REPLACE VIEW chairman.silver.vw_flights_analyst AS
SELECT
  flight_id,
  market,
  origin,
  destination,
  fare_eur,
  status,
  pax_name,
  chairman.security.mask_email(pax_email) AS pax_email,
  chairman.security.mask_credit_card(credit_card) AS pax_credit_card
FROM chairman.silver.flights
WHERE chairman.security.rf_only_eu_or_engineers(market);
```

> Copia el siguiente contenido en una celda

```python
select *
from chairman.silver.vw_flights_analyst
```

##### 2.7.5.3 Damos permisos para entrar a la tabla (GRANT)

Importante: las políticas (row filter / masks) no sustituyen los GRANT.

Primero necesitas el derecho a leer (SELECT),

y después las políticas modulan lo que ves (filas/columnas).

Para escribir, usamos MODIFY (engloba las operaciones de DML usuales).

> Copia el siguiente contenido en una celda

```python
# Engineers: lectura + modificación (INSERT/UPDATE/DELETE/TRUNCATE/MERGE)

spark.sql(f"GRANT SELECT ON VIEW chairman.silver.vw_flights_analyst TO UC_Data_Engineer")
```

# Unity Catalog: Data Discovery

## Bloque 3: IA Documentación + Tagging

## 3.1 Generar Documentación con IA


En Databricks es posible **generar documentación automática de las columnas de una tabla usando IA integrada**.  

Esto permite obtener **nombres descriptivos, definiciones y explicaciones de cada campo** sin hacerlo manualmente, lo que **acelera y simplifica enormemente** el proceso de documentación y gobernanza de datos.


%md
##### Vamos a documentar la tabla `sesion2.silver.dim_aeronaves`.

1. Dirígete a **Catalog**.  
2. Navega hasta: **sesion2 → silver → dim_aeronaves**.  
3. Una vez dentro de la tabla, haz clic en **AI Generate** para generar automáticamente la descripción de las columnas.



<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_docu1.png" width="1600px"/>


Una vez que la IA haya generado la descripción para cada columna, revisa el contenido y haz clic en **Save all** para guardar toda la documentación en la tabla.


%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_docu2.png" width="600px"/>

El resultado debería ser similar a la siguiente captura: todas las columnas de la tabla aparecerán ahora **documentadas con su descripción**, generadas automáticamente mediante IA.

%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_docu3.png" width="1600px"/>

## 3.2 Tags en UC



Coloca Tags para **facilitar la busqueda de datos y definir politicas**


- Puedes asignar **tags** a catálogos, esquemas, tablas y columnas.
- Útil para clasificar PII, datos financieros, confidenciales, internos, etc.
- Databricks puede incluso **sugerir tags automáticamente usando IA**.
- Estos tags pueden ser usados para **automatizar políticas** (ej. aplicar máscara si `tag = PII`).
- Luego es posible buscar datasets por tag → *“ver todos los datos certificados y financieros”*.

### 3.2.1 Crear un Governed Tag en Unity Catalog

En primer lugar, dirígete a  
**Catalog → Governance → Governed Tags**


%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags1.png" width="1600px"/>

**Crear una nueva etiqueta gobernada** -> Una vez dentro de la pantalla de **Governed Tags**, haz clic en  
**Create a governed tag** para crear una nueva etiqueta.



<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags2.png" width="1600px"/>


Rellena los campos para crear el primer tag con las siguientes características:

🔸 **Tag Name:** `Business_domain`  
🔹 **Allowed values:** `Finance`, `Marketing`, `Logistics`
 


<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags3.png" width="1200px"/>

Ahora, crea ADICIONALMENTE **dos governed tags** con esta configuración:

🔸 **Tag Name:** `Business_sensibility`  
🔹 **Allowed values:** `High`, `Medium`, `Low`

🔸 **Tag Name:** `PII`  
🔹 **Allowed values:** `PII_1`, `PII_2`

---

Si haces clic en **Permissions** de cualquier Tag, podrás ver qué usuarios o grupos tienen permiso para **usar, asignar o gestionar** ese Tag.  
Los tags deben **habilitarse explícitamente** para los usuarios o grupos que podrán aplicarlos a tablas o columnas.


%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags4.png" width="1600px"/>

### 3.2.2 Asignar Tag


Navega dentro del catálogo hasta la tabla  
**sesion2 → bronze → dim_aeronaves**  
y haz clic en **Add tags** para asignar las etiquetas creadas.


%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags5.png" width="1600px"/>

Aplica los siguientes Governed Tags:

🔹 A la **tabla completa** (sesion2.**bronze**.dim_aeronaves):
- Business_sensibility = High  
- Business_domain = Finance

🔹 A la **columna** fabricante:
- PII = PII_1

🔹 A la **tabla completa** (sesion2.**silver**.dim_aeronaves):
- Business_sensibility = Medium  
- Business_domain = Finance

%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags6.png" width="1600px"/>

### 3.2.3 Buscar Tablas por Tags

%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags7.png" width="1600px"/>

%md

<img src="https://raw.githubusercontent.com/jmartinezceste/IEF_Course1_360degreesDatabricks/main/photos/UC/sesion3_Tags8.png" width="1600px"/>

Aunque la búsqueda desde la interfaz del **Catalog** solo funciona para objetos como tablas, vistas, funciones y volúmenes,  
podemos localizar **programáticamente** aquellas tablas que tienen **columnas etiquetadas con un determinado tag**, consultando las vistas de `system.information_schema.column_tags`.


```python
SELECT
  schema_name,
  table_name,
  column_name,
  tag_name,
  tag_value
FROM system.information_schema.column_tags
WHERE tag_name = 'PII'
  AND tag_value = 'PII_1';
```

Esto puede ser útil para aplicar operaciones automatizadas, como **encriptación, anonimización, enmascarado o auditoría**, sobre todas las columnas que tengan una etiqueta específica (por ejemplo, `PII`, `High`, `Sensitive`), sin tener que identificarlas manualmente una por una.


In [0]:
# 1️⃣ Obtener columnas etiquetadas PII_1
pii_columns = spark.sql("""
    SELECT
      catalog_name,
      schema_name,
      table_name,
      column_name
    FROM system.information_schema.column_tags
    WHERE tag_name = 'PII'
      AND tag_value = 'PII 1'
""").collect()

# ⚠️ Si no hay columnas con ese tag, detener
if not pii_columns:
    raise Exception("No hay columnas con tag PII_1.")

# Obtenemos catálogo y tabla (asumiendo que todas son de la misma tabla)
catalog = pii_columns[0]["catalog_name"]
schema= pii_columns[0]["schema_name"]
table_name = pii_columns[0]["table_name"]

display(catalog, schema, table_name)



'sesion2'

'bronze'

'dim_aeronaves'

In [0]:
# Listado de columnas marcadas con tag
masked_cols = [row["column_name"] for row in pii_columns]

# 2️⃣ Generar SELECT dinámico
columns = spark.table(f"{catalog}.{schema}.{table_name}").columns

select_exprs = [
    f"chairman.security.mask_email({col}) AS {col}" if col in masked_cols else col
    for col in columns
]

select_sql = ",\n  ".join(select_exprs)

# 3️⃣ Crear la vista final
final_sql = f"""
CREATE OR REPLACE VIEW sesion2.silver.vw_flights_masked AS
SELECT
  {select_sql}
FROM {catalog}.{schema}.{table_name};
"""

print(final_sql)
spark.sql(final_sql)


CREATE OR REPLACE VIEW sesion2.silver.vw_flights_masked AS
SELECT
  aeronave_id,
  modelo,
  chairman.security.mask_email(fabricante) AS fabricante,
  anio_fabricacion,
  ingest_time
FROM sesion2.bronze.dim_aeronaves;



DataFrame[]

In [0]:
%sql
select * from sesion2.silver.vw_flights_masked

aeronave_id,modelo,fabricante,anio_fabricacion,ingest_time
A001,B777,Boeing,2013,2025-11-27T08:38:04.889Z
A002,B737,Boeing,2013,2025-11-27T08:38:04.889Z
A003,E190,Embraer,2020,2025-11-27T08:38:04.889Z
A004,B737,Boeing,2013,2025-11-27T08:38:04.889Z
A005,B777,Boeing,2007,2025-11-27T08:38:04.889Z
A006,A350,Airbus,2012,2025-11-27T08:38:04.889Z
A007,A350,Airbus,2015,2025-11-27T08:38:04.889Z
A008,A350,Airbus,2022,2025-11-27T08:38:04.889Z
A009,A320,Airbus,2009,2025-11-27T08:38:04.889Z
A010,B737,Boeing,2012,2025-11-27T08:38:04.889Z
